In [4]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
# Load the stockist data
stockist_df = pd.read_csv("stockist_data.csv")


In [6]:
# Define product columns
product_columns = [
    "AIS(Air Insulated Switchgear)", "RMU(Ring Main Unit)", "PSS(Compact Sub-Stations)",
    "VCU(Vacuum Contactor Units)", "E-House", "VCB(Vacuum Circuit Breaker)",
    "ACB(Air Circuit Breaker)", "MCCB(Moduled Case Circuit Breaker)", "SDF(Switch Disconnectors)",
    "BBT(Busbar Trunking)", "Modular Switches", "Starter", "Controller",
    "Solar Solutions", "Pump Starter and Controller"
]

In [7]:
# Extract the user-product matrix
user_product_matrix = stockist_df.set_index("Partner_id")[product_columns]
user_product_matrix = user_product_matrix.astype(int)  # Convert to numeric


In [8]:
# Split data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(user_product_matrix, test_size=0.2, random_state=42)


In [9]:
# Fit KNN Model for User-Based Collaborative Filtering on training data
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(train_data)


NearestNeighbors(algorithm='brute', metric='cosine')

In [10]:
# Function to recommend products with similarity scores
def recommend_products_with_scores(partner_id, n_neighbors=5):
    if partner_id not in user_product_matrix.index:
        return [], []


    # Find similar users
    #distances, indices = knn.kneighbors([user_product_matrix.loc[partner_id]], n_neighbors=n_neighbors+1)
    distances, indices = knn.kneighbors(user_product_matrix.loc[[partner_id]], n_neighbors=n_neighbors+1)

    # Exclude self (first index)
    similar_users = user_product_matrix.iloc[indices[0][1:]]
    similarity_scores = 1 - distances[0][1:]  # Convert distance to similarity

    # Compute weighted sum of product purchases
    recommended_products = similar_users.T.dot(similarity_scores).sort_values(ascending=False)

    # Filter out already purchased products
    already_purchased = user_product_matrix.loc[partner_id]
    recommended_products = recommended_products[~already_purchased.astype(bool)]

    # Select top 3 recommendations
    top_recommendations = list(recommended_products.head(3).index)
    top_similarity_scores = list(recommended_products.head(3).values)

    return top_recommendations, top_similarity_scores


In [ ]:
# Generate recommendations with similarity scores
recommendations_with_scores = []
for partner_id in stockist_df["Partner_id"]:
    recommended_products, similarity_scores = recommend_products_with_scores(partner_id)
    recommendations_with_scores.append([partner_id, recommended_products, similarity_scores])


In [ ]:
# Convert to DataFrame
user_based_recommendations_df = pd.DataFrame(recommendations_with_scores, columns=["Partner_ID", "Recommended_Products", "Similarity_Scores"])


In [ ]:
# Save recommendations to CSV
output_file = "User_Based_Recommendations.csv"
user_based_recommendations_df.to_csv(output_file, index=False)


In [ ]:
print(f"User-based collaborative filtering recommendations saved to {output_file}")